# TCMNP R-Language資料庫 使用Python查詢範例 (Windows)

## 1. 安裝R和Python (建立新環境時)

R-Language https://www.r-project.org/

RTools https://cran.r-project.org/bin/windows/Rtools/

Python https://www.python.org/

### 一次複製一行，貼在R Studio內並執行

if(!require(devtools))install.packages("devtools")

if(!require(TCMNP))devtools::install_github("tcmlab/TCMNP",upgrade = FALSE,dependencies = TRUE)

## 2. Python套件、R套件

安裝Python套件

In [1]:
!pip install -Ur requirements.txt

載入需要的Python套件

In [2]:
import pandas as pd

## To convert Traditional Chinese to Simplified Chinese
from opencc import OpenCC
cc = OpenCC('tw2sp')

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

## To aid in printing HTML in notebooks
import rpy2.ipython.html
rpy2.ipython.html.init_printing()

## To see plots in an output cell
# from rpy2.ipython.ggplot import image_png

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.
C:\Users\eavic\AppData\Local\Programs\Python\Python313\Lib\site-packages\rpy2\rinterface\__init__.py:1211: UserWarning: Environment variable "PATH" redefined by R and overriding existing variable. Current: "C:\Program Files (x86)\VMware\VMware Workstation\bin\;C:\Program Files\Microsoft MPI\Bin\;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\libnvvp;c:\windows\system32;c:\windows;c:\windows\system32\wbem;c:\windows\system32\windowspowershell\v1.0\;c:\windows\system32\openssh\;c:\program files\dotnet\;c:\program files\git\cmd;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\Nsight Compute 2023.1.1\;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\NVIDIA Corporation\

載入需要的R套件 (如果沒安裝則會自動安裝)

In [3]:
robjects.r('''
if(!require(arrow))install.packages("arrow")
if(!require(devtools))install.packages("devtools")
if(!require(TCMNP))devtools::install_github("tcmlab/TCMNP",upgrade = FALSE,dependencies = TRUE)
''')

R callback write-console: Loading required package: arrow
  
R callback write-console: 
Attaching package: 'arrow'

  
R callback write-console: The following object is masked from 'package:utils':

    timestamp

  
R callback write-console: Loading required package: devtools
  
R callback write-console: Loading required package: usethis
  
R callback write-console: Loading required package: TCMNP
  


## 3. 以中藥查詢作用基因&分子

繁體中文轉簡體中文

中藥名填在cht_herbs內

In [4]:
cht_herbs = ['麻黃', '甘草','苦杏仁','石膏', '薏苡仁', '蒼朮', '青蒿', '豬苓', '馬鞭草', '葶藶子','橘紅', '虎杖', '廣藿香','蘆根']
chs_herbs = []
for herb in cht_herbs:
    chs_herbs.append(cc.convert(herb))
herbs = str(chs_herbs)[1:-1]  # 去頭去尾

查詢，如果有不在資料庫內的中藥會有訊息提醒

In [5]:
rpy2_df = robjects.r(f'''
herbs <- c({herbs})
herb_target(herbs, type = "Herb_cn_name")
''')

[1] "石膏 is/are not in the datasets."


把查詢到的 R DataFrame 轉換成 Pandas DataFrame

In [6]:
with (robjects.default_converter + pandas2ri.converter).context():
    df = robjects.conversion.get_conversion().rpy2py(rpy2_df)
type(df)

pandas.core.frame.DataFrame

資料列標題

In [7]:
df.columns

Index(['herb', 'molecule_id', 'molecule', 'target'], dtype='object')

資料表大小(行數, 列數)

In [8]:
df.shape

(10937, 4)

開頭5筆資料

In [9]:
df.head()

,herb,molecule_id,molecule,target
1,cang zhu,wogonin,tcm01563,NOS2
2,cang zhu,wogonin,tcm01563,PTGS1
3,cang zhu,wogonin,tcm01563,ESR1
4,cang zhu,wogonin,tcm01563,AR
5,cang zhu,wogonin,tcm01563,SCN5A


匯出成Excel(CSV會亂碼)

In [10]:
df.to_excel('1_中藥查詢.xlsx')

##  4. 以基因查詢中藥

基因名稱填在target_genes內

In [11]:
target_genes = ['MAPK1', 'JUN', 'FOS', 'RAC1', 'IL1', 'IL6']
genes = str(target_genes)[1:-1]  # 去頭去尾

查詢，沒在資料庫內的基因會有訊息提醒

In [12]:
rpy2_df2 = robjects.r(f'''
gene <- c({genes})
target_herb(gene)
''')

[1] "IL1 is/are not in the datasets."


把查詢到的 R DataFrame 轉換成 Pandas DataFrame

In [13]:
with (robjects.default_converter + pandas2ri.converter).context():
    df2 = robjects.conversion.get_conversion().rpy2py(rpy2_df2)
type(df2)

pandas.core.frame.DataFrame

資料列標題

In [14]:
df2.columns

Index(['herb', 'molecule_id', 'molecule', 'target'], dtype='object')

資料表大小(行數, 列數)

In [15]:
df2.shape

(2365, 4)

開頭5筆資料

In [16]:
df2.head()

,herb,molecule_id,molecule,target
1,ai di cha,kaempferol,tcm00154,JUN
2,ai di cha,quercetin,tcm00192,FOS
3,ai di cha,quercetin,tcm00192,MAPK1
4,ai di cha,quercetin,tcm00192,JUN
5,ai di cha,quercetin,tcm00192,IL6


匯出成Excel(CSV會亂碼)

In [17]:
df2.to_excel('2_基因查詢.xlsx')

## 5. 資料庫內所有基因靶點的列表

In [18]:
with (robjects.default_converter + pandas2ri.converter).context():
    rpy2_df3 = robjects.r('''disease_data''')
    df3 = robjects.conversion.get_conversion().rpy2py(rpy2_df3)
df3

,.
1,RPL17-C18orf32
2,RPL36A-HNRNPH2
3,RPS10-NUDT3
4,LOC102723407
5,ADAR
...,...
786,SLC39A8
787,RABGAP1L
788,TGM2
789,AGER


## 6. 利用疾病靶點基因查詢中藥及其處方，[[1]]：中藥 [[2]]：方劑

In [19]:
with (robjects.default_converter + pandas2ri.converter).context():
    rpy2_df4 = robjects.r('''
    tcm_prescription(disease_data)[[1]]
    ''')
    df4 = robjects.conversion.get_conversion().rpy2py(rpy2_df4)
    df4.to_excel('3_基因查詢中藥.xlsx')
df4

,Herb_cn_name,freq
1,人参,437
2,甘草,433
3,胡芦巴,325
4,皂角刺,288
5,地榆,276
...,...,...
269,肉豆蔻,53
270,浙贝母,53
271,草果,51
272,赤芍,51


In [20]:
with (robjects.default_converter + pandas2ri.converter).context():
    rpy2_df5 = robjects.r('''
    tcm_prescription(disease_data)[[2]]
    ''')
    df5 = robjects.conversion.get_conversion().rpy2py(rpy2_df5)
    df5.replace({'&nbsp;': ' '}, regex=True, inplace=True)  # 取代NBSP不換行空白標籤，修正匯出時造成易讀性的問題
    df5.to_excel('4_基因查詢方劑.xlsx')
df5

,CompoundId,Pvalue,OddsRatio,ExpCount,Count,Size,p.adjust,herb
1,sheng yang yi wei tang,0.09939,1.45,6.22,9.0,13.0,0.516,黄芪/半夏/人参/防风/白芍/羌活/茯苓/柴胡/黄连
2,bai du san,0.03958,1.67,4.78,8.0,10.0,0.430,柴胡/前胡/枳壳/羌活/茯苓/桔梗/人参/甘草
3,chai ge jie ji tang （ tao shi ）,0.00259,2.09,3.82,8.0,8.0,0.115,柴胡/葛根/甘草/黄芩/羌活/白芷/白芍/桔梗
4,jing fang bai du san,0.03958,1.67,4.78,8.0,10.0,0.430,羌活/柴胡/前胡/枳壳/茯苓/荆芥/防风/桔梗
5,cang lin san,0.22507,1.33,5.26,7.0,11.0,0.668,人参/茯苓/前胡/羌活/桔梗/枳壳/柴胡
...,...,...,...,...,...,...,...,...
85,hua shi bai du fang,0.00093,1.93,6.22,12.0,13.0,0.083,麻黄/苦杏仁/甘草/苍术/草果/半夏/茯苓/黄芪/葶苈子/赤芍/广藿香/大黄
86,qing fei pai du tang,0.12953,1.32,9.08,12.0,19.0,0.549,麻黄/苦杏仁/茯苓/柴胡/黄芩/半夏/紫菀/款冬花/射干/山药/枳实/广藿香
87,chai ge jie ji tang,0.01247,1.74,5.74,10.0,12.0,0.277,柴胡/葛根/黄芩/羌活/白芷/白芍/桔梗/赤芍/知母/牡丹皮
88,xuan fei bai du fang,0.01247,1.74,5.74,10.0,12.0,0.277,麻黄/苦杏仁/苍术/广藿香/青蒿/马鞭草/葶苈子/化橘红/甘草/虎杖


## 7. 自己建立疾病標靶點基因查詢中藥和方劑 (R會報錯)

建立查詢列表，模仿內建資料集的結構

In [21]:
search_gene_targets = {'.': ['ADAR', 'RPL36A-HNRNPH2', 'AGER']}
search_df = pd.DataFrame(search_gene_targets, index=None)
search_df

,.
0,ADAR
1,RPL36A-HNRNPH2
2,AGER


In [22]:
with (robjects.default_converter + pandas2ri.converter).context():
    rpy2_search_df = robjects.conversion.get_conversion().py2rpy(search_df)
rpy2_search_df

,,.
0,0,ADAR
1,1,RPL36A-HNRNPH2
2,2,AGER


In [23]:
robjects.r.assign('rpy2_search_df', rpy2_search_df)
robjects.r('''
row.names(rpy2_search_df) <- NULL
rpy2_search_df
''')

,,.
0,1,ADAR
1,2,RPL36A-HNRNPH2
2,3,AGER
